# Database Read and Write Examples for R

Notes:

1. RJDBC: https://cran.r-project.org/web/packages/RJDBC/RJDBC.pdf
2. This notebook has been tested in R 3.6 Notebook in CPD 3.5.

<a class="anchor" id="toc"></a>
## TOC:
1. [Create db connection using RJDBC](#dbcon)   
2. [Common SQL Actions](#sql)
* [2.1 Query tables in three steps: write, send, fetch](#sql1)
* [2.2 Write tables](#sql2)
* [2.3 Update tables](#sql3)
* [2.4 Delete tables](#sql4)

3. [Close db connection](#close)   

4. [Appendix: Read and save data to local project](#projectlib)

<a class="anchor" id="dbcon"></a>
## 1. Create db connection using RJDBC
[Top](#toc)



### Use Insert to Code 

**The following cell was generated by Insert to Code(R DataFrame) menu. You can either regenerate it for Connected Data defined in your project or modify this code.**

If you are not regenerating code, make sure to verify or change:

1. Connection name
2. Schema name
3. Table name

*Look for "To Do" tag in the code*

In [70]:
# @hidden_cell

library(projectLib)
project <- projectLib::Project$new()

# To Do: change or verify connection name. It should match the connection name that you defined in your project
DB2_Cloud_metadata = project$get_connection(name="***")

library(RJDBC)

drv <- JDBC(driverClass="com.ibm.db2.jcc.DB2Driver", classPath="/opt/jdbc/db2jcc4.jar", identifier.quote = "\"")

DB2_Cloud_url <- paste("jdbc:db2://",
    DB2_Cloud_metadata[][["host"]],
    ":", "50000",
    "/", DB2_Cloud_metadata[][["database"]],
    sep=""
)

DB2_Cloud_connection <- dbConnect(drv,
    DB2_Cloud_url,
    DB2_Cloud_metadata[][["username"]],
    DB2_Cloud_metadata[][["password"]]
)

# To Do: change to your own table
#query <- "SELECT * FROM \"TLR48624\".\"CUSTOMER_CHURN\""
query <- "SELECT * FROM TLR48624.CUSTOMER_CHURN"
data <- dbSendQuery(DB2_Cloud_connection, query)
# fetch first 5 rows
data_df_1 <- dbFetch(data, n = 5)
head(data_df_1)


Gender,Status,Children,Est Income,Car Owner,Age,AvgMonthlySpend,CustomerSupportCalls,Paymethod,MembershipPlan,CHURN
M,S,1,57626.9,Y,43,38.96,0,CC,2,F
F,M,1,57063.0,Y,52,81.30,0,CH,4,F
M,S,2,100020.0,N,50,293.24,0,CH,4,T
F,M,2,84789.3,N,50,10.45,0,CH,4,T
M,M,1,97180.4,N,50,45.66,0,CC,4,T


 <a class="anchor" id="sql"></a>
## 2. Common SQL Actions
[Top](#toc)


 <a class="anchor" id="sql1"></a>
### 2.1 Query tables in three steps: write, send, fetch

In [236]:
# To Do: change to your own table
# write SQL query
query <- "SELECT * FROM jpf66625.CREDITDATA"
# send query
data <- dbSendQuery(DB2_Cloud_connection, query)

# fetch results
data_df_1 <- dbFetch(data)

head(data_df_1)

Column_0,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose
0,67,male,2,own,NA,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NA,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car
5,35,male,1,free,NA,NA,9055,36,education


 <a class="anchor" id="sql2"></a>
### 2.2 Write tables
[Top](#toc)

Using iris as an example.

In [250]:
data(iris)

In [251]:
# To Do: change to your own table
# write tables
dbWriteTable(DB2_Cloud_connection, "jpf66625.RTABLE", iris)

[1] TRUE

In [252]:
# To Do: change to your own table
# check if table has been created successfully
# write SQL query
query <- "SELECT * FROM jpf66625.RTABLE"

# send query
data <- dbSendQuery(DB2_Cloud_connection, query)

# fetch results
data_df_1 <- dbFetch(data)

head(data_df_1)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,SPECIES
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


 <a class="anchor" id="sql3"></a>
## 2.3 Update tables
[Top](#toc)

### 1. using **dbSendUpdate**

In [253]:
# To Do: change to your own table
# using dbSendUpdate
query <- "INSERT INTO jpf66625.RTABLE VALUES (5.1,3.5,1.4, 0.2, 'test')"
data <- dbSendUpdate(DB2_Cloud_connection, query)

In [254]:
# To Do: change to your own table
# check if insert works
query <- "SELECT * FROM jpf66625.RTABLE WHERE SPECIES = 'test'"
data <- dbSendQuery(DB2_Cloud_connection, query)
data_df_1 <- dbFetch(data)
data_df_1

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,SPECIES
5.1,3.5,1.4,0.2,test


### 2. using **dbWriteTable** with append option

In [255]:
dim(iris)

[1] 150   5

In [256]:
# To Do: change to your own table
# Note: dont include schema for this line aka. use RTABLE instead of jpf66625.RTABLE
dbWriteTable(DB2_Cloud_connection, 'RTABLE', iris, append = TRUE, overwrite= F)

[1] TRUE

In [257]:
# To Do: change to your own table
# check if append works: it should be 150 + 150 + 1 = 301
query <- "SELECT * FROM jpf66625.RTABLE"
data <- dbSendQuery(DB2_Cloud_connection, query)
data_df_1 <- dbFetch(data)
dim(data_df_1)

[1] 301   5

 <a class="anchor" id="sql4"></a>
## 2.4 Delete tables
[Top](#toc)


In [260]:
# To Do: change to your own table
query <- "DROP TABLE jpf66625.RTABLE"
data <- dbSendUpdate(DB2_Cloud_connection, query)

In [261]:
# To Do: change to your own table
# dont include schema for this line aka. use RTABLE instead of jpf66625.RTABLE
dbExistsTable(DB2_Cloud_connection, "RTABLE")

[1] FALSE

<a class="anchor" id="close"></a>
## 3. Close db connection
[Top](#toc)


In [35]:
# After use, close the database connection with the following code:
dbDisconnect(DB2_Cloud_connection)

[1] TRUE

<a class="anchor" id="projectlib"></a>
## Appendix: saving to local project

[Top](#toc)

In [42]:
library(projectLib)
#project <- projectLib::Project$new()
project <- access_project()

### Read data

In [46]:
my.file <- project$get_file("model output summary.csv")
df.data <-  read.csv(text = rawToChar(my.file))
head(df.data)

CUSTOMER_ID,PROD_ECOLOGICAL,PROD_MEAT_SUBST,PROD_FAIRTRADE,PROD_REFORM,GREEN_ENERGY,FREQUENTING_NATURE,ENVIR_CHARITY,NATURE_CHARITY,AGE_GROUP,...,TOTAL_USAGE_YEAR1,TOTAL_USAGE_YEAR2,ENERGY_SAVING,COST_TO_SERVE,NUMBER_OF_QUESTIONS_ANSWERED_YES,NUMBER_OF_QUESTIONS_ANSWERED_CAT,LIFESTYLE_CLUSTER,CUSTOMER_ENGAGEMENT,LIFESTYLE_CLUSTER_DESCRIPTION,CUSTOMER_ENGAGEMENT_CLUSTER_DESCRIPTION
1,1,0,0,0,1,0,0,0,<20,...,5055.71,3979.552,-21.286000,75.05,2,TWO OR THREE,3,5,Green Neutral,"High Energy Saving, Low Cost-to-Serve"
2,0,0,0,1,1,0,0,0,35 to 39,...,8636.82,7077.181,-18.058019,145.17,2,TWO OR THREE,3,3,Green Neutral,"High Energy Saving, Medium to High Cost-to-Serve"
3,0,1,0,0,0,1,1,1,>64,...,11454.37,12733.784,11.169658,164.59,4,FOUR OR MORE,2,1,Green Positive,"Medium Energy Saving, High Cost-to-Serve"
4,0,1,0,0,0,1,0,0,30 to 34,...,5668.38,5208.521,-8.112700,144.40,2,TWO OR THREE,3,3,Green Neutral,"High Energy Saving, Medium to High Cost-to-Serve"
5,0,1,1,0,0,0,0,0,30 to 34,...,7499.15,9032.882,20.452081,67.64,2,TWO OR THREE,3,2,Green Neutral,"Medium Energy Saving, Low Cost-to-Serve"
6,0,0,0,0,0,0,0,0,20 to 24,...,11026.90,11490.816,4.207133,61.60,0,ONE OR LESS,1,2,Green Negative,"Medium Energy Saving, Low Cost-to-Serve"


### Save data

In [45]:
csv_lines <- capture.output(write.csv(df.data, row.names=FALSE), type="output")
csv_raw <- charToRaw(paste0(csv_lines, collapse='\n'))
project$save_data("file.csv", csv_raw)

$file_name
[1] "file.csv"

$message
[1] "File saved to project storage."

$asset_id
[1] "b892cc9e-3ccc-4c31-adc2-2ec62448f1ee"